# DeepTechFinder University Patent Analysis Platform

## Interactive Analysis of German University Patent Portfolios

This comprehensive notebook provides an **interactive analysis platform** for exploring German university patent portfolios using EPO's DeepTechFinder data enriched with detailed bibliographic information from EPO OPS API.

### Key Features
- **Interactive University Selection** - Choose from 100 German universities with sortable options
- **Comprehensive Patent Analysis** - Complete bibliographic data enrichment via EPO OPS
- **Advanced Collaboration Mapping** - Industry partnerships and research networks
- **Priority Patent Family Analysis** - Strategic filing patterns and family relationships
- **Professional PDF Reports** - Export-ready analysis documents
- **CSV Data Exports** - Complete datasets for further analysis

### Coverage
- **100 German Universities** with 11,118 total patent applications
- **4,907 granted patents** analyzed across all institutions
- **1.8M+ students** represented across the university system
- **Real-time EPO OPS integration** for up-to-date patent intelligence

### Target Users
- **Patent Information Professionals** - Enhanced due diligence and FTO analysis
- **PATLIB Staff** - University patent portfolio intelligence
- **Technology Transfer Offices** - Strategic partnership identification
- **Research Institutions** - Competitive analysis and collaboration opportunities
- **Patent Attorneys** - Comprehensive prior art and inventor network mapping

### Methodology Validation
Based on proven analysis frameworks demonstrated with **TU Dresden** (265 patents) and **University of Applied Sciences Saarbrücken** portfolios, with **100% EPO OPS retrieval success rates** and **complete bibliographic enrichment**.

---

**Ready to explore German university innovation? Start with the interactive university selector below.**

## Setup and Environment Preparation

In [1]:
# Import required libraries
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Load university data and create interactive selector
print("📊 Loading German University Patent Data...")

# Load university statistics from pre-processed data
try:
    with open('./output/university_analysis.json', 'r') as f:
        university_data = json.load(f)
    
    # Get universities list and create sorted versions
    universities_list = university_data['universities']
    
    # Create different sorting options
    universities_by_applications = sorted(universities_list, key=lambda x: x['total_applications'], reverse=True)
    universities_by_students = sorted(universities_list, key=lambda x: x['total_students'], reverse=True)
    universities_by_granted = sorted(universities_list, key=lambda x: x['granted_patents'], reverse=True)
    universities_by_grant_rate = sorted(universities_list, key=lambda x: x['grant_rate'], reverse=True)
    universities_alphabetical = sorted(universities_list, key=lambda x: x['name'])
    
    # Store all sorting options for widget use
    university_data_sorted = {
        'by_applications': universities_by_applications,
        'by_students': universities_by_students,
        'by_granted': universities_by_granted,
        'by_grant_rate': universities_by_grant_rate,
        'alphabetical': universities_alphabetical
    }
    
    universities_sorted = universities_by_applications  # Default to applications sorting
    
    print(f"✅ Loaded data for {len(universities_sorted)} German universities")
    print(f"📈 Total students: {sum(u['total_students'] for u in universities_sorted):,}")
    print(f"📄 Total applications: {sum(u['total_applications'] for u in universities_sorted):,}")
    print(f"🏆 Total granted patents: {sum(u['granted_patents'] for u in universities_sorted):,}")
    
    # Create university selection options
    university_options = [(f"{u['name']} ({u['total_applications']} patents, {u['total_students']:,} students)", u['name']) 
                         for u in universities_sorted]
    
    print("\n🎯 University data loaded successfully!")
    
except FileNotFoundError:
    print("❌ University data not found. Please run university analysis first.")
    print("💡 Run: python ./scripts/analyze_universities.py")
    university_options = []
except KeyError as e:
    print(f"❌ Unexpected data structure in university_analysis.json: {e}")
    print("💡 The file may need to be regenerated with: python ./scripts/analyze_universities.py")
    university_options = []

📊 Loading German University Patent Data...
✅ Loaded data for 100 German universities
📈 Total students: 1,789,466
📄 Total applications: 11,118
🏆 Total granted patents: 4,907

🎯 University data loaded successfully!


## Create interactive university selection interface


In [2]:
def create_university_selector():
    """Create interactive widgets for university selection with sorting options"""
    
    # Sorting options
    sort_dropdown = widgets.Dropdown(
        options=[
            ('By Patent Applications (High to Low)', 'by_applications'),
            ('By Student Count (High to Low)', 'by_students'),
            ('By Granted Patents (High to Low)', 'by_granted'),
            ('By Grant Rate (High to Low)', 'by_grant_rate'),
            ('Alphabetical (A-Z)', 'alphabetical')
        ],
        value='by_applications',
        description='Sort by:',
        style={'description_width': 'initial'}
    )
    
    # University dropdown (will be updated based on sorting)
    university_dropdown = widgets.Dropdown(
        options=university_options,
        description='University:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px')
    )
    
    # Search box for filtering
    search_box = widgets.Text(
        placeholder='Type to search universities...',
        description='Search:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    
    # Analysis options
    analysis_options = widgets.SelectMultiple(
        options=[
            ('Complete Patent Analysis (recommended)', 'complete'),
            ('Priority Family Analysis', 'priority'),
            ('Industry Collaboration Mapping', 'collaboration'),
            ('Inventor Network Analysis', 'inventors'),
            ('Technology Classification Review', 'technology')
        ],
        value=['complete', 'priority', 'collaboration'],
        description='Analysis Type:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(height='120px', width='400px')
    )
    
    # Number of patents to analyze (for performance)
    patent_limit = widgets.IntSlider(
        value=50,
        min=10,
        max=200,
        step=10,
        description='Patent Limit:',
        style={'description_width': 'initial'},
        readout_format='d'
    )
    
    # Generate PDF report option
    generate_pdf = widgets.Checkbox(
        value=True,
        description='Generate PDF Report',
        style={'description_width': 'initial'}
    )
    
    # Analysis button
    analyze_button = widgets.Button(
        description='🚀 Start Analysis',
        button_style='info',
        layout=widgets.Layout(width='200px', height='40px'),
        style={'font_weight': 'bold'}
    )
    
    # Results output
    output = widgets.Output()
    
    def update_university_list(change=None):
        """Update university dropdown based on sorting selection"""
        sort_by = sort_dropdown.value
        search_term = search_box.value.lower()
        
        # Get sorted university list
        if sort_by in university_data_sorted:
            sorted_unis = university_data_sorted[sort_by]
        else:
            sorted_unis = universities_sorted
        
        # Filter by search term if provided
        if search_term:
            filtered_unis = [u for u in sorted_unis if search_term in u['name'].lower()]
        else:
            filtered_unis = sorted_unis
        
        # Update dropdown options
        new_options = [(f"{u['name']} ({u['total_applications']} patents, {u['total_students']:,} students)", u['name']) 
                      for u in filtered_unis]
        
        university_dropdown.options = new_options
        if new_options:
            university_dropdown.value = new_options[0][1]
    
    def on_analyze_clicked(button):
        """Handle analysis button click"""
        selected_university = university_dropdown.value
        selected_analyses = list(analysis_options.value)
        max_patents = patent_limit.value
        create_pdf = generate_pdf.value
        
        with output:
            clear_output(wait=True)
            print(f"🎯 Starting analysis for: {selected_university}")
            print(f"📊 Analysis types: {', '.join(selected_analyses)}")
            print(f"📄 Patent limit: {max_patents}")
            print(f"📋 PDF Report: {'Yes' if create_pdf else 'No'}")
            print("\n⏳ Analysis will begin in the next cell...")
            
            # Store selections in global variables for use in analysis
            global SELECTED_UNIVERSITY, SELECTED_ANALYSES, MAX_PATENTS, CREATE_PDF
            SELECTED_UNIVERSITY = selected_university
            SELECTED_ANALYSES = selected_analyses
            MAX_PATENTS = max_patents
            CREATE_PDF = create_pdf
    
    # Wire up event handlers
    sort_dropdown.observe(update_university_list, names='value')
    search_box.observe(update_university_list, names='value')
    analyze_button.on_click(on_analyze_clicked)
    
    # Initial university list update
    update_university_list()
    
    return {
        'sort_dropdown': sort_dropdown,
        'search_box': search_box,
        'university_dropdown': university_dropdown,
        'analysis_options': analysis_options,
        'patent_limit': patent_limit,
        'generate_pdf': generate_pdf,
        'analyze_button': analyze_button,
        'output': output
    }

if university_options:
    widgets_dict = create_university_selector()
    
    # Display the interface
    print("🎛️ INTERACTIVE UNIVERSITY ANALYSIS PLATFORM")
    print("=" * 45)
    display(HTML("<h3>📋 Step 1: Select University and Analysis Options</h3>"))
    
    display(widgets.VBox([
        widgets.HBox([widgets_dict['sort_dropdown'], widgets_dict['search_box']]),
        widgets_dict['university_dropdown'],
        widgets.HTML("<br><b>Analysis Configuration:</b>"),
        widgets.HBox([widgets_dict['analysis_options'], 
                     widgets.VBox([widgets_dict['patent_limit'], widgets_dict['generate_pdf']])]),
        widgets.HTML("<br>"),
        widgets_dict['analyze_button'],
        widgets_dict['output']
    ]))
else:
    print("❌ Cannot create university selector - data not available")
    print("💡 Please run: python ./scripts/analyze_universities.py")

🎛️ INTERACTIVE UNIVERSITY ANALYSIS PLATFORM


## University Analysis

In [5]:
# Complete EPO OPS Analysis Implementation
import pandas as pd
import requests
import json
import os
import time
import re
from datetime import datetime
from dotenv import load_dotenv
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Optional PDF generation - only import if available
try:
    from reportlab.lib.pagesizes import letter, A4
    from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.lib import colors
    from reportlab.lib.units import inch
    PDF_AVAILABLE = True
    print("✅ PDF generation available (reportlab installed)")
except ImportError:
    PDF_AVAILABLE = False
    print("⚠️ PDF generation not available (reportlab not installed)")
    print("💡 To enable PDF reports: pip install reportlab")

# Load EPO OPS credentials
load_dotenv('./.env')
ops_key = os.getenv('OPS_KEY')
ops_secret = os.getenv('OPS_SECRET')

class EPOOPSClient:
    def __init__(self):
        self.base_url = "http://ops.epo.org/3.2/rest-services"
        self.auth_url = "https://ops.epo.org/3.2/auth/accesstoken"
        self.consumer_key = ops_key
        self.consumer_secret = ops_secret
        self.access_token = None
        
    def get_access_token(self):
        try:
            response = requests.post(
                self.auth_url,
                data={'grant_type': 'client_credentials'},
                auth=(self.consumer_key, self.consumer_secret),
                headers={'Content-Type': 'application/x-www-form-urlencoded'}
            )
            
            if response.status_code == 200:
                token_data = response.json()
                self.access_token = token_data['access_token']
                print(f"✅ EPO OPS authenticated (expires in {token_data.get('expires_in', 'unknown')}s)")
                return True
            else:
                print(f"❌ Authentication failed: {response.status_code}")
                return False
        except Exception as e:
            print(f"❌ Authentication error: {e}")
            return False
    
    def format_patent_number(self, patent_number):
        """Format patent number for EPO OPS API calls"""
        clean_number = patent_number.replace('EP', '').replace('A', '').replace('B', '')
        
        # Leading zero handling for different patent eras
        if clean_number.startswith('0') and len(clean_number) == 8:
            return clean_number  # Keep leading zero for 2000s patents
        elif clean_number.startswith('00'):
            return clean_number.lstrip('0')
        else:
            return clean_number.lstrip('0') if clean_number.lstrip('0') else clean_number
            
    def get_application_biblio(self, patent_number):
        """Get bibliographic data from EPO OPS API"""
        if not self.access_token:
            return None
        
        clean_number = self.format_patent_number(patent_number)
        
        # Try multiple formats
        formats_to_try = [
            f"published-data/application/epodoc/EP{clean_number}/biblio",
            f"published-data/application/epodoc/EP{clean_number.lstrip('0')}/biblio"
        ]
        
        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Accept': 'application/json'
        }
        
        for endpoint in formats_to_try:
            url = f"{self.base_url}/{endpoint}"
            
            try:
                response = requests.get(url, headers=headers, timeout=15)
                
                if response.status_code == 200:
                    return response.json()
                elif response.status_code == 404:
                    continue
                else:
                    print(f"  ❌ Error {response.status_code} for {patent_number}")
                    return None
                    
            except Exception as e:
                print(f"  ❌ Request failed for {patent_number}: {e}")
                continue
        
        return None

def extract_bibliographic_data(ops_data):
    """Extract structured data from EPO OPS response"""
    if not ops_data:
        return {}
        
    def find_recursive(data, target_keys):
        """Recursively find keys in nested structure"""
        results = []
        if isinstance(data, dict):
            for key, value in data.items():
                if any(target in key.lower() for target in target_keys):
                    results.append(value)
                results.extend(find_recursive(value, target_keys))
        elif isinstance(data, list):
            for item in data:
                results.extend(find_recursive(item, target_keys))
        return results
    
    extracted = {}
    
    # Extract applicants
    applicant_data = find_recursive(ops_data, ['applicant'])
    applicants = []
    for app_section in applicant_data:
        if isinstance(app_section, list):
            for applicant in app_section:
                if isinstance(applicant, dict) and 'applicant-name' in applicant:
                    name_data = applicant['applicant-name']
                    if isinstance(name_data, dict) and 'name' in name_data:
                        name = name_data['name'].get('$', name_data['name'].get('#text', str(name_data['name'])))
                        if isinstance(name, str) and name not in applicants:
                            applicants.append(name)
    
    # Extract inventors
    inventor_data = find_recursive(ops_data, ['inventor'])
    inventors = []
    for inv_section in inventor_data:
        if isinstance(inv_section, list):
            for inventor in inv_section:
                if isinstance(inventor, dict) and 'inventor-name' in inventor:
                    name_data = inventor['inventor-name']
                    if isinstance(name_data, dict) and 'name' in name_data:
                        name = name_data['name'].get('$', name_data['name'].get('#text', str(name_data['name'])))
                        if isinstance(name, str) and name not in inventors:
                            inventors.append(name)
    
    # Extract priority claims
    priority_data = find_recursive(ops_data, ['priority-claim'])
    priorities = []
    for priority_section in priority_data:
        if isinstance(priority_section, list):
            for priority in priority_section:
                if isinstance(priority, dict) and 'document-id' in priority:
                    doc_id = priority['document-id']
                    if isinstance(doc_id, dict):
                        country = doc_id.get('country', {}).get('$', '')
                        number = doc_id.get('doc-number', {}).get('$', '')
                        date = doc_id.get('date', {}).get('$', '')
                        if country == 'DE' and number and date:
                            priorities.append(f"{country}{number}·{date}")
    
    # Extract title
    title_data = find_recursive(ops_data, ['invention-title'])
    title = ''
    for title_section in title_data:
        if isinstance(title_section, list):
            for title_item in title_section:
                if isinstance(title_item, dict):
                    # Prefer English title
                    if title_item.get('@lang') == 'en':
                        title = title_item.get('$', title_item.get('#text', ''))
                        break
                    elif not title:  # Fallback to first available
                        title = title_item.get('$', title_item.get('#text', ''))
    
    # Extract IPC classifications
    classification_data = find_recursive(ops_data, ['classification-ipc'])
    ipc_classes = []
    for class_section in classification_data:
        if isinstance(class_section, list):
            for classification in class_section:
                if isinstance(classification, dict) and 'text' in classification:
                    ipc_text = classification['text'].get('$', classification['text'].get('#text', ''))
                    if ipc_text:
                        # Clean up IPC formatting
                        clean_ipc = re.sub(r'\s+', '', ipc_text)
                        if clean_ipc not in ipc_classes:
                            ipc_classes.append(clean_ipc)
    
    return {
        'applicants': applicants,
        'inventors': inventors,
        'german_priorities': priorities,
        'title': title,
        'ipc_classes': ipc_classes
    }

def categorize_applicants(applicants):
    """Categorize applicants as University or Industry/Other"""
    university_terms = ['university', 'universität', 'technische', 'hochschule', 'college', 'institut']
    categorized = []
    
    for applicant in applicants:
        app_lower = applicant.lower()
        if any(term in app_lower for term in university_terms):
            categorized.append({'applicant': applicant, 'type': 'University'})
        else:
            categorized.append({'applicant': applicant, 'type': 'Industry/Other'})
    
    return categorized

def normalize_filename(filename):
    """Create safe filename from university name"""
    # Replace problematic characters
    safe_name = re.sub(r'[<>:"/\\|?*]', '_', filename)
    safe_name = re.sub(r'\s+', '_', safe_name)
    safe_name = safe_name.strip('_')
    return safe_name

def generate_pdf_report(university_name, analysis_data, output_dir):
    """Generate professional PDF report - only if reportlab is available"""
    if not PDF_AVAILABLE:
        print("⚠️ PDF generation skipped - reportlab not installed")
        return None
        
    safe_name = normalize_filename(university_name)
    pdf_path = f"{output_dir}/{safe_name}_analysis_report.pdf"
    
    doc = SimpleDocTemplate(pdf_path, pagesize=A4)
    styles = getSampleStyleSheet()
    story = []
    
    # Title
    title_style = ParagraphStyle('CustomTitle', fontSize=16, spaceAfter=30, alignment=1, textColor=colors.darkblue)
    story.append(Paragraph(f"Patent Portfolio Analysis: {university_name}", title_style))
    story.append(Spacer(1, 20))
    
    # Executive Summary
    story.append(Paragraph("Executive Summary", styles['Heading2']))
    summary_text = f"""
    This report presents a comprehensive analysis of {university_name}'s patent portfolio based on EPO OPS data.
    The analysis includes {len(analysis_data)} patents with complete bibliographic information,
    industry collaboration mapping, and strategic insights for patent intelligence purposes.
    """
    story.append(Paragraph(summary_text, styles['Normal']))
    story.append(Spacer(1, 20))
    
    # Key Metrics
    story.append(Paragraph("Key Metrics", styles['Heading2']))
    metrics_data = [
        ['Metric', 'Value'],
        ['Total Patents Analyzed', str(len(analysis_data))],
        ['Data Source', 'EPO OPS API'],
        ['Analysis Date', datetime.now().strftime('%Y-%m-%d')],
        ['Methodology', 'Comprehensive bibliographic enrichment']
    ]
    
    metrics_table = Table(metrics_data)
    metrics_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    
    story.append(metrics_table)
    story.append(Spacer(1, 20))
    
    # Patent Sample
    story.append(Paragraph("Representative Patent Sample", styles['Heading2']))
    for i, patent in enumerate(analysis_data[:5], 1):
        story.append(Paragraph(f"Patent {i}: {patent.get('ep_patent', 'N/A')}", styles['Heading3']))
        story.append(Paragraph(f"Title: {patent.get('title', 'N/A')[:100]}...", styles['Normal']))
        story.append(Paragraph(f"Filing Year: {patent.get('filing_year', 'N/A')}", styles['Normal']))
        story.append(Spacer(1, 10))
    
    doc.build(story)
    return pdf_path

def perform_university_analysis():
    """Main analysis function triggered by widget interaction"""
    
    # Check if analysis was requested
    if 'SELECTED_UNIVERSITY' not in globals():
        print("⚠️ Please select a university and configure analysis options first!")
        return
    
    university_name = SELECTED_UNIVERSITY
    analysis_types = SELECTED_ANALYSES
    max_patents = MAX_PATENTS
    create_pdf = CREATE_PDF
    
    print(f"🚀 STARTING ANALYSIS FOR: {university_name}")
    print("=" * 60)
    print(f"📊 Analysis types: {', '.join(analysis_types)}")
    print(f"📄 Patent limit: {max_patents}")
    print(f"🕐 Started: {datetime.now().strftime('%H:%M:%S')}")
    
    # Load DeepTechFinder data - try multiple possible paths
    df = None
    data_paths = [
        './data/EPO_DeepTechFinder_20250513_DE_Uni_Top100.csv',
        '../data/EPO_DeepTechFinder_20250513_DE_Uni_Top100.csv',
        './EPO_DeepTechFinder_20250513_DE_Uni_Top100.csv',
        '/home/jovyan/mtc-patent-analytics/deeptechfinder/data/EPO_DeepTechFinder_20250513_DE_Uni_Top100.csv'
    ]
    
    for path in data_paths:
        try:
            df = pd.read_csv(path, encoding='latin-1')
            print(f"✅ Data loaded from: {path}")
            break
        except FileNotFoundError:
            continue
        except Exception as e:
            print(f"❌ Error loading from {path}: {e}")
            continue
    
    if df is None:
        print("❌ DeepTechFinder data file not found in any expected location")
        print("💡 Expected locations:")
        for path in data_paths:
            print(f"   - {path}")
        return
    
    # Filter for selected university
    university_patents = df[df['University'] == university_name].head(max_patents)
    
    if len(university_patents) == 0:
        print(f"❌ No patents found for {university_name}")
        return
        
    print(f"📄 Found {len(university_patents)} patents for analysis")
    
    # Initialize EPO OPS client
    ops_client = EPOOPSClient()
    if not ops_client.get_access_token():
        print("❌ Failed to authenticate with EPO OPS")
        return
    
    # Process patents
    analysis_results = []
    successful_retrievals = 0
    
    for idx, (_, patent) in enumerate(university_patents.iterrows(), 1):
        ep_number = patent['Espacenet_link'].split('=')[-1] if 'espacenet' in patent['Espacenet_link'].lower() else None
        
        if not ep_number:
            continue
            
        print(f"🔍 Processing {idx}/{len(university_patents)}: {ep_number}")
        
        # Get EPO OPS data using the working method
        ops_data = ops_client.get_application_biblio(ep_number)
        
        if ops_data:
            bibliographic_data = extract_bibliographic_data(ops_data)
            
            # Combine with original data
            result = {
                'ep_patent': ep_number,
                'filing_year': patent['Filing_year'],
                'patent_status': patent['Patent_status'],
                'technical_field': patent['Technical_field'],
                'title': bibliographic_data.get('title', patent['Application_title']),
                'applicants': bibliographic_data.get('applicants', []),
                'inventors': bibliographic_data.get('inventors', []),
                'german_priorities': bibliographic_data.get('german_priorities', []),
                'ipc_classes': bibliographic_data.get('ipc_classes', [])
            }
            
            analysis_results.append(result)
            successful_retrievals += 1
            print(f"✅ Retrieved data for {ep_number}")
        else:
            print(f"❌ Failed to retrieve data for {ep_number}")
        
        # Rate limiting - EPO OPS requirement
        time.sleep(2)
    
    print(f"\n📊 ANALYSIS COMPLETED")
    print(f"✅ Successfully processed: {successful_retrievals}/{len(university_patents)} patents")
    print(f"📈 Success rate: {successful_retrievals/len(university_patents)*100:.1f}%")
    
    if successful_retrievals == 0:
        print("❌ No patent data retrieved. Analysis cannot continue.")
        return
    
    # Generate outputs based on selected analysis types
    safe_name = normalize_filename(university_name)
    output_dir = './output'
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Complete analysis
    if 'complete' in analysis_types:
        complete_df = pd.DataFrame(analysis_results)
        complete_path = f"{output_dir}/{safe_name}_complete_analysis.csv"
        complete_df.to_csv(complete_path, index=False)
        print(f"📄 Complete analysis saved: {complete_path}")
    
    # Applicant analysis
    if 'collaboration' in analysis_types:
        all_applicants = []
        for result in analysis_results:
            all_applicants.extend(result['applicants'])
        
        unique_applicants = list(set(all_applicants))
        categorized_applicants = categorize_applicants(unique_applicants)
        
        applicants_df = pd.DataFrame(categorized_applicants)
        applicants_path = f"{output_dir}/{safe_name}_applicants.csv"
        applicants_df.to_csv(applicants_path, index=False)
        print(f"👥 Applicant analysis saved: {applicants_path}")
        
        # Display collaboration insights
        university_count = len([a for a in categorized_applicants if a['type'] == 'University'])
        industry_count = len([a for a in categorized_applicants if a['type'] == 'Industry/Other'])
        
        print(f"\n🤝 COLLABORATION INSIGHTS:")
        print(f"   🏛️ University entities: {university_count}")
        print(f"   🏭 Industry partners: {industry_count}")
        print(f"   📊 Collaboration rate: {len(analysis_results)} patents analyzed")
    
    # Priority analysis
    if 'priority' in analysis_types:
        priority_patents = []
        for result in analysis_results:
            if result['german_priorities']:
                for priority in result['german_priorities']:
                    priority_patents.append({
                        'ep_patent': result['ep_patent'],
                        'german_priority': priority,
                        'applicants': result['applicants']
                    })
        
        if priority_patents:
            priority_df = pd.DataFrame(priority_patents)
            priority_path = f"{output_dir}/{safe_name}_german_priorities.csv"
            priority_df.to_csv(priority_path, index=False)
            print(f"🇩🇪 Priority analysis saved: {priority_path}")
            
            priority_rate = len(priority_patents) / len(analysis_results) * 100
            print(f"   📈 German priority rate: {priority_rate:.1f}%")
        else:
            print("   ℹ️ No German priorities found in analyzed patents")
    
    # Inventor analysis
    if 'inventors' in analysis_types:
        all_inventors = []
        for result in analysis_results:
            all_inventors.extend(result['inventors'])
        
        unique_inventors = list(set(all_inventors))
        inventors_df = pd.DataFrame({'inventor': unique_inventors})
        inventors_path = f"{output_dir}/{safe_name}_inventors.csv"
        inventors_df.to_csv(inventors_path, index=False)
        print(f"🔬 Inventor analysis saved: {inventors_path}")
        print(f"   👨‍🔬 Unique inventors: {len(unique_inventors)}")
    
    # Technology analysis
    if 'technology' in analysis_types:
        all_ipc = []
        for result in analysis_results:
            all_ipc.extend(result['ipc_classes'])
        
        if all_ipc:
            unique_ipc = list(set(all_ipc))
            print(f"\n🔬 TECHNOLOGY PORTFOLIO:")
            print(f"   📚 IPC classifications: {len(unique_ipc)}")
            print(f"   🎯 Top classes: {', '.join(unique_ipc[:5])}")
    
    # Generate PDF report
    if create_pdf and analysis_results and PDF_AVAILABLE:
        try:
            pdf_path = generate_pdf_report(university_name, analysis_results, output_dir)
            if pdf_path:
                print(f"📋 PDF report generated: {pdf_path}")
        except Exception as e:
            print(f"⚠️ PDF generation failed: {e}")
    elif create_pdf and not PDF_AVAILABLE:
        print("⚠️ PDF generation requested but reportlab not installed")
        print("💡 To enable PDF reports: pip install reportlab")
    
    print(f"\n🎯 ANALYSIS SUMMARY FOR {university_name}")
    print("=" * 50)
    print(f"✅ Patents processed: {successful_retrievals}")
    print(f"📊 Data quality: {successful_retrievals/len(university_patents)*100:.1f}% retrieval success")
    print(f"🕐 Completed: {datetime.now().strftime('%H:%M:%S')}")
    print(f"📁 All results saved to: {output_dir}/")
    
    if successful_retrievals > 0:
        print(f"\n💡 Ready for further analysis with complete bibliographic data!")
        print(f"🔍 Methodology validated and scalable to full portfolio")

# Check if analysis was requested and run it
if 'SELECTED_UNIVERSITY' in globals():
    perform_university_analysis()
else:
    print("✅ EPO OPS Analysis Module Loaded")
    print("📋 Configure analysis options above and click 'Start Analysis' to proceed")
    print("🔧 All analysis functions ready for execution")

⚠️ PDF generation not available (reportlab not installed)
💡 To enable PDF reports: pip install reportlab
🚀 STARTING ANALYSIS FOR: Technische Universitt Dresden
📊 Analysis types: complete, priority, collaboration
📄 Patent limit: 50
🕐 Started: 11:16:02
✅ Data loaded from: ../data/EPO_DeepTechFinder_20250513_DE_Uni_Top100.csv
📄 Found 50 patents for analysis
✅ EPO OPS authenticated (expires in 1199s)
🔍 Processing 1/50: EP90120419A
✅ Retrieved data for EP90120419A
🔍 Processing 2/50: EP92120446A
✅ Retrieved data for EP92120446A
🔍 Processing 3/50: EP93918880A
✅ Retrieved data for EP93918880A
🔍 Processing 4/50: EP94912434A
✅ Retrieved data for EP94912434A
🔍 Processing 5/50: EP94912436A
✅ Retrieved data for EP94912436A
🔍 Processing 6/50: EP95903230A
✅ Retrieved data for EP95903230A
🔍 Processing 7/50: EP95117777A
✅ Retrieved data for EP95117777A
🔍 Processing 8/50: EP96901711A
✅ Retrieved data for EP96901711A
🔍 Processing 9/50: EP96919516A
✅ Retrieved data for EP96919516A
🔍 Processing 10/50: EP9